[View in Colaboratory](https://colab.research.google.com/github/abhibee/tensorflow/blob/master/Detect_emotions_fer2013.ipynb)

In [1]:
import sys, os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import regularizers




Using TensorFlow backend.


In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
!mkdir models
!pwd
f = h5py.File("model.h5", mode='a')
f.close()


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
/content


In [45]:
"""f = h5py.File("model.h5", mode='a')
f.close()"""

'f = h5py.File("model.h5", mode=\'a\')\nf.close()'

In [4]:
!pwd
!ls
BASEPATH = 'drive/colab/EmotionRecognition'
sys.path.insert(0, BASEPATH)
os.chdir(BASEPATH)
MODELPATH = 'model.h5'
!ls fer2013-small*

/content
datalab  drive	model.h5  models
fer2013-small.csv


In [0]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

In [0]:
data = pd.read_csv('fer2013.csv')

In [7]:
data.tail()


,emotion,pixels,Usage
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest
35886,2,19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...,PrivateTest


In [0]:
pixels = data['pixels'].tolist() # 1

faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] # 2
    face = np.asarray(face).reshape(width, height) # 3
    face = face / 255.0 # 4
    face = cv2.resize(face.astype('uint8'), (width, height)) # 5
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1) # 6

emotions = pd.get_dummies(data['emotion']).as_matrix() # 7

In [0]:
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

In [0]:
from keras import regularizers

model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 128)       512       
__________

In [0]:
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'], options = run_opts)

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)

In [0]:
tensorboard = TensorBoard(log_dir='./logs')

In [0]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')

In [0]:
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

In [0]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

Train on 29068 samples, validate on 3589 samples
Epoch 1/100
29068/29068 [==============================] - 70s 2ms/step - loss: 2.0817 - acc: 0.2117 - val_loss: 1.8379 - val_acc: 0.2455

Epoch 00001: val_loss improved from inf to 1.83793, saving model to model.h5
Epoch 2/100
 5888/29068 [=====>........................] - ETA: 52s - loss: 1.8655 - acc: 0.2366

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8472 - acc: 0.2420 - val_loss: 1.8218 - val_acc: 0.2455

Epoch 00002: val_loss improved from 1.83793 to 1.82185, saving model to model.h5
Epoch 3/100
14528/29068 [=============>................] - ETA: 32s - loss: 1.8353 - acc: 0.2449

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8285 - acc: 0.2486 - val_loss: 1.8227 - val_acc: 0.2455

Epoch 00003: val_loss did not improve from 1.82185
Epoch 4/100
18816/29068 [==================>...........] - ETA: 23s - loss: 1.8179 - acc: 0.2521

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8186 - acc: 0.2513 - val_loss: 1.8106 - val_acc: 0.2488

Epoch 00004: val_loss improved from 1.82185 to 1.81059, saving model to model.h5
Epoch 5/100
18048/29068 [=================>............] - ETA: 24s - loss: 1.8093 - acc: 0.2527

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8121 - acc: 0.2526 - val_loss: 1.8077 - val_acc: 0.2455

Epoch 00005: val_loss improved from 1.81059 to 1.80769, saving model to model.h5
Epoch 6/100
17856/29068 [=================>............] - ETA: 25s - loss: 1.8128 - acc: 0.2508

...

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8042 - acc: 0.2542 - val_loss: 1.8118 - val_acc: 0.2530

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.0008100000384729356.

Epoch 00011: val_loss did not improve from 1.80769
Epoch 12/100
15808/29068 [===============>..............] - ETA: 29s - loss: 1.8021 - acc: 0.2561

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8015 - acc: 0.2535 - val_loss: 1.8327 - val_acc: 0.2455

Epoch 00012: val_loss did not improve from 1.80769
Epoch 13/100
19136/29068 [==================>...........] - ETA: 22s - loss: 1.7999 - acc: 0.2564

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8009 - acc: 0.2549 - val_loss: 1.8022 - val_acc: 0.2527

Epoch 00013: val_loss improved from 1.80769 to 1.80221, saving model to model.h5
Epoch 14/100
18112/29068 [=================>............] - ETA: 24s - loss: 1.7998 - acc: 0.2578

29068/29068 [==============================] - 68s 2ms/step - loss: 1.8006 - acc: 0.2551 - val_loss: 1.8142 - val_acc: 0.2458

Epoch 00014: val_loss did not improve from 1.80221
Epoch 15/100
19840/29068 [===================>..........] - ETA: 20s - loss: 1.7976 - acc: 0.2589

27584/29068 [===========================>..] - ETA: 3s - loss: 1.7985 - acc: 0.2567

In [0]:
scores = model.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))